In [1]:
# coding: utf-8
%load_ext autoreload
%autoreload 2

In [2]:
import os

import pandas as pd

import h1st as h1
h1.init()

from examples.RayTune.sklearn_smv_classifier import SklearnSVMClassifier

from examples.RayTune import config
from examples.RayTune.utils import prepare_train_test_data

In [3]:
import ray
from ray import tune

ray.init(ignore_reinit_error=True)

2020-10-26 13:38:30,815	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '192.168.1.2',
 'raylet_ip_address': '192.168.1.2',
 'redis_address': '192.168.1.2:10699',
 'object_store_address': '/tmp/ray/session_2020-10-26_13-38-30_246451_70653/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-10-26_13-38-30_246451_70653/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2020-10-26_13-38-30_246451_70653',
 'metrics_export_port': 64879}

## Class H1stTune

In [17]:
import json
import os    

from filelock import FileLock 

from examples.RayTune.tensorflow_mlp_classifier import TensorflowMLPClassifier

# class H1stTune(tune.Trainable):

#     h1st_ml_model = None
#     parameters = {}
    
#     def setup(self, config):
#         from filelock import FileLock 
#         self.config = config
#         self.timestep = 0

model_class = TensorflowMLPClassifier

class TrainH1stModel(tune.Trainable):    
    def setup(self, config):
        self.timestep = 0
#         self.config = config
#         self.timestep = 0
        self.kwargs = {}
        self.hyperparameter = getattr(model_class, 'hyperparameter', {})['tuning_param']
        for k, v in self.hyperparameter.items():
            self.kwargs[k] = v(config[k])
#         kwargs = {
#             'units': int(config['units']),
#             'epochs': int(config['epochs']), 
#             'lr': config['lr'], 
#             'n_layer': int(config['n_layer'])
#         }

        self.h1_ml_model = model_class(**self.kwargs)
#         self.h1_ml_model = model_class(
#             units=int(config['units']), 
#             epochs=int(config['epochs']), 
#             lr=config['lr'], 
#             n_layer=int(config['n_layer']))

#     TensorflowMLPClassifier(
#             units=int(config['units']), 
#             epochs=int(config['epochs']), 
#             lr=config['lr'], 
#             n_layer=int(config['n_layer']))
        
        DATA_ROOT = './data/credit_card'
        lock_file = f'{DATA_ROOT}/data.lock'
        if not os.path.exists(DATA_ROOT):
            os.makedirs(DATA_ROOT)
        with FileLock(os.path.expanduser(lock_file)):
            data = self.h1_ml_model.load_data()
        self.prepared_data = self.h1_ml_model.prep(data)  
        
    def step(self):
        self.timestep += 1
        self.h1_ml_model.train(self.prepared_data)
        self.h1_ml_model.evaluate(self.prepared_data)
        acc = self.h1_ml_model.metrics['accuracy']
        self.h1_ml_model.persist(str(self.timestep) + str())
        return {"mean_accuracy": acc}

    def save_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        with open(path, "w") as f:
            f.write(json.dumps({"timestep": self.timestep}))
        return path

    def load_checkpoint(self, checkpoint_path):
        with open(checkpoint_path) as f:
            self.timestep = json.loads(f.read())["timestep"]



hyperparameter = {
    'tuning_param': {
        'lr': float, # 0.01 ~ 0.1
        'units': int # 4 ~ 16
    },
    'target_metrics': {
        'accuracy': 'minimize'
    }
}

            
class TrainH1stModel2(tune.Trainable):
#     def __init__(self, h1st_model):
#         self.h1_tf_mlp = h1st_model
    def setup(self, config):
        self.timestep = 0
        self.config = config
        self.h1_tf_mlp = TensorflowMLPClassifier(
            units=int(config['units']), 
            epochs=int(config['epochs']), 
            lr=config['lr'], 
            n_layer=int(config['n_layer']))
        
        DATA_ROOT = './data/credit_card'
        lock_file = f'{DATA_ROOT}/data.lock'
        if not os.path.exists(DATA_ROOT):
            os.makedirs(DATA_ROOT)
        with FileLock(os.path.expanduser(lock_file)):
            data = self.h1_tf_mlp.load_data()
        self.prepared_data = self.h1_tf_mlp.prep(data)  
        
    def step(self):
        self.timestep += 1
        self.h1_tf_mlp.train(self.prepared_data)
        self.h1_tf_mlp.evaluate(self.prepared_data)
        acc = self.h1_tf_mlp.metrics['accuracy']
        self.h1_tf_mlp.persist(str(self.timestep) + str())
        return {"mean_accuracy": acc}

    def save_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        with open(path, "w") as f:
            f.write(json.dumps({"timestep": self.timestep}))
        return path

    def load_checkpoint(self, checkpoint_path):
        with open(checkpoint_path) as f:
            self.timestep = json.loads(f.read())["timestep"]
            
            
class MyTrainableClass(tune.Trainable):
    """Example agent whose learning curve is a random sigmoid.

    The dummy hyperparameters "width" and "height" determine the slope and
    maximum reward value reached.
    """

    def setup(self, config):
        self.timestep = 0

    def step(self):
        self.timestep += 1
        v = np.tanh(float(self.timestep) / self.config.get("width", 1))
        v *= self.config.get("height", 1)

        # Here we use `episode_reward_mean`, but you can also report other
        # objectives such as loss or accuracy.
        return {"episode_reward_mean": v}

    def save_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        with open(path, "w") as f:
            f.write(json.dumps({"timestep": self.timestep}))
        return path

    def load_checkpoint(self, checkpoint_path):
        with open(checkpoint_path) as f:
            self.timestep = json.loads(f.read())["timestep"]            

In [23]:
tmp_model = TensorflowMLPClassifier(
            units=32, 
            epochs=5, 
            lr=0.01, 
            n_layer=5)

In [24]:
tmp_model.hyperparameter

{'tuning_param': {'lr': float, 'units': int},
 'target_metrics': {'accuracy': 'minimize'}}

## Population Base Training

In [6]:
# config = {
#     "lr": tune.grid_search([0.005]),
#     "units": tune.grid_search([16, 32, 64, 128]),
#     "n_layer": tune.grid_search([3, 5, 10])
# #     "n_layer": tune.uniform(1, 10)
# }

# analysis = tune.run(
#     TrainH1stModel, 
#     config=config, 
#     verbose=2,
#     stop={"training_iteration": 3}
# )

In [21]:
from ray.tune.schedulers import PopulationBasedTraining

experiment_metrics = dict(metric="mean_accuracy", mode="max")

pbt_scheduler = PopulationBasedTraining(
        time_attr='training_iteration',
        perturbation_interval=2,  # Every N time_attr units, "perturb" the parameters.
        hyperparam_mutations={           
            "lr": [0.005],
            "epochs": [2, 10, 30, 100],
            "units": [4, 8, 16, 32, 64],
            "n_layer": [2, 4, 6]            
        })

# Note: This appears to be needed to avoid a "key error", but in fact these values won't change
# in the analysis.dataframe() object, even though they will be tuned by the PBT scheduler.
# So when you look at the analysis.dataframe(), look at the `experiment_tag` to see the actual values!
config = {
    "lr": 0.005,            # Use the lowest values from the previous definition
    "units": 16,
    "n_layer": 4,
    "epochs": 2
}

In [22]:
# analysis = tune.run(TrainH1stModel, 
#     scheduler=pbt_scheduler, 
#     config=config,
#     stop={"training_iteration": 4},
# #     stop={"mean_accuracy": 0.82, "training_iteration": 200},                    
#     num_samples=40,
#     verbose=1,
#     metric="mean_accuracy",
#     mode="max"
# )

analysis= tune.run(
    TrainH1stModel2,
    config=config,
    metric="mean_accuracy",
    mode="max",
    scheduler=pbt_scheduler,
    stop={"training_iteration": 2},
    verbose=1,
    num_samples=10
)

(pid=76289) INFO:examples.RayTune.tensorflow_mlp_classifier:(9900, 1), (9900, 1)
(pid=76289) 2020-10-26 15:51:48,786 INFO h1st.model_repository.model_repository: Saving metrics property...
(pid=76289) INFO:h1st.model_repository.model_repository:Saving metrics property...
(pid=76289) 2020-10-26 15:51:48,787 INFO h1st.model_repository.model_repository: Saving stats property...
(pid=76289) INFO:h1st.model_repository.model_repository:Saving stats property...
(pid=76289) 2020-10-26 15:51:48,787 INFO h1st.model_repository.model_repository: Saving model property...
(pid=76289) INFO:h1st.model_repository.model_repository:Saving model property...
(pid=76290) INFO:examples.RayTune.tensorflow_mlp_classifier:(9900, 1), (9900, 1)
(pid=76290) 2020-10-26 15:51:48,799 INFO h1st.model_repository.model_repository: Saving metrics property...
(pid=76290) INFO:h1st.model_repository.model_repository:Saving metrics property...
(pid=76290) 2020-10-26 15:51:48,799 INFO h1st.model_repository.model_repository: S

Trial name,status,loc,acc,iter,total time (s)
TrainH1stModel2_ca67f_00000,TERMINATED,,0.800505,2,1.89468
TrainH1stModel2_ca67f_00001,TERMINATED,,0.796465,2,1.88865
TrainH1stModel2_ca67f_00002,TERMINATED,,0.799596,2,1.89827
TrainH1stModel2_ca67f_00003,TERMINATED,,0.80202,2,1.91395
TrainH1stModel2_ca67f_00004,TERMINATED,,0.802828,2,1.89794
TrainH1stModel2_ca67f_00005,TERMINATED,,0.800202,2,1.91203
TrainH1stModel2_ca67f_00006,TERMINATED,,0.803232,2,1.6625
TrainH1stModel2_ca67f_00007,TERMINATED,,0.802828,2,1.67947
TrainH1stModel2_ca67f_00008,TERMINATED,,0.802727,2,1.68189
TrainH1stModel2_ca67f_00009,TERMINATED,,0.797879,2,1.67298


(pid=76290) INFO:h1st.model_repository.model_repository:Saving model property...
(pid=76334) INFO:examples.RayTune.tensorflow_mlp_classifier:(9900, 1), (9900, 1)
(pid=76334) 2020-10-26 15:51:48,786 INFO h1st.model_repository.model_repository: Saving metrics property...
(pid=76334) INFO:h1st.model_repository.model_repository:Saving metrics property...
(pid=76334) 2020-10-26 15:51:48,786 INFO h1st.model_repository.model_repository: Saving stats property...
(pid=76334) INFO:h1st.model_repository.model_repository:Saving stats property...
(pid=76334) 2020-10-26 15:51:48,787 INFO h1st.model_repository.model_repository: Saving model property...
(pid=76334) INFO:h1st.model_repository.model_repository:Saving model property...


In [14]:
analysis.dataframe()[['mean_accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False).head(20)

,mean_accuracy,config/epochs,config/lr,config/n_layer,config/units
104,0.820101,2,0.005,4,16
135,0.820000,2,0.005,4,16
14,0.819394,2,0.005,4,16
193,0.819192,2,0.005,4,16
74,0.818990,2,0.005,4,16
155,0.818990,2,0.005,4,16
15,0.818990,2,0.005,4,16
53,0.818990,2,0.005,4,16
19,0.818889,2,0.005,4,16
81,0.818687,2,0.005,4,16


In [15]:
analysis.dataframe()[['mean_accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False).head(20).mean_accuracy.mean()

0.8186666666666668

In [16]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

 993.47 seconds,   16.56 minutes


## BO Hyperband (BOHB)

In [ ]:
# import json
# import os

# import numpy as np

# import ray
# from ray import tune
# from ray.tune import Trainable
# from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
# from ray.tune.suggest.bohb import TuneBOHB


# class MyTrainableClass(Trainable):
#     """Example agent whose learning curve is a random sigmoid.

#     The dummy hyperparameters "width" and "height" determine the slope and
#     maximum reward value reached.
#     """

#     def setup(self, config):
#         self.timestep = 0

#     def step(self):
#         self.timestep += 1
#         v = np.tanh(float(self.timestep) / self.config.get("width", 1))
#         v *= self.config.get("height", 1)

#         # Here we use `episode_reward_mean`, but you can also report other
#         # objectives such as loss or accuracy.
#         return {"episode_reward_mean": v}

#     def save_checkpoint(self, checkpoint_dir):
#         path = os.path.join(checkpoint_dir, "checkpoint")
#         with open(path, "w") as f:
#             f.write(json.dumps({"timestep": self.timestep}))
#         return path

#     def load_checkpoint(self, checkpoint_path):
#         with open(checkpoint_path) as f:
#             self.timestep = json.loads(f.read())["timestep"]


# if __name__ == "__main__":
#     import ConfigSpace as CS  # noqa: F401
#     ray.init(num_cpus=8)

#     config = {
#         "iterations": 100,
#         "width": tune.uniform(0, 20),
#         "height": tune.uniform(-100, 100),
#         "activation": tune.choice(["relu", "tanh"])
#     }

#     # Optional: Pass the parameter space yourself
#     # config_space = CS.ConfigurationSpace()
#     # config_space.add_hyperparameter(
#     #     CS.UniformFloatHyperparameter("width", lower=0, upper=20))
#     # config_space.add_hyperparameter(
#     #     CS.UniformFloatHyperparameter("height", lower=-100, upper=100))
#     # config_space.add_hyperparameter(
#     #     CS.CategoricalHyperparameter(
#     #         "activation", choices=["relu", "tanh"]))

#     experiment_metrics = dict(metric="episode_reward_mean", mode="max")

#     bohb_hyperband = HyperBandForBOHB(
#         time_attr="training_iteration",
#         max_t=100,
#         reduction_factor=4,
#         **experiment_metrics)

#     bohb_search = TuneBOHB(
#         # space=config_space,  # If you want to set the space manually
#         max_concurrent=4,
#         **experiment_metrics)

#     tune.run(
#         MyTrainableClass,
#         name="bohb_test",
#         config=config,
#         scheduler=bohb_hyperband,
#         search_alg=bohb_search,
#         num_samples=10,
#         stop={"training_iteration": 100})

In [5]:
import ConfigSpace as CS
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB

In [6]:
config_space = CS.ConfigurationSpace()

# There are also UniformIntegerHyperparameter and UniformFloatHyperparameter
# objects for defining integer and float ranges, respectively. For example:
# config_space.add_hyperparameter(
#     CS.UniformIntegerHyperparameter('foo', lower=0, upper=100))

config_space.add_hyperparameter(
    CS.UniformFloatHyperparameter('lr', lower=0.005, upper=0.05))
config_space.add_hyperparameter(
    CS.UniformIntegerHyperparameter('units', lower=1, upper=64))
config_space.add_hyperparameter(
    CS.UniformIntegerHyperparameter('n_layer', lower=1, upper=8))
config_space.add_hyperparameter(
    CS.UniformIntegerHyperparameter('epochs', lower=4, upper=50))


# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter('lr', choices=[0.005]))
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter('units', choices=[4, 8, 16, 32]))
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter('n_layer', choices=[2, 4, 6, 8]))
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter('epochs', choices=[2, 10, 30, 100]))

epochs, Type: UniformInteger, Range: [4, 50], Default: 27

In [9]:
experiment_metrics = dict(metric="mean_accuracy", mode="max")
algo = TuneBOHB(config_space, max_concurrent=10, **experiment_metrics)
# algo = TuneBOHB(config_space, max_concurrent=10)
# search_algorithm = TuneBOHB(config_space, max_concurrent=4)

scheduler = HyperBandForBOHB(
    time_attr='training_iteration',
    reduction_factor=4)

In [10]:
analysis= tune.run(
    TrainH1stModel,
    metric="mean_accuracy",
    mode="max",     
    search_alg=algo,
    scheduler=scheduler,
    stop={"training_iteration": 5},
    verbose=1,
    num_samples=200
)

# analysis = tune.run(
#     TrainH1stModel, 
#     scheduler=scheduler, 
#     search_alg=search_algorithm, 
#     metric="mean_accuracy",
#     mode="max",         
#     num_samples=40,                           # Force it try all 12 combinations
#     stop={"training_iteration": 4},
#     verbose=1
# )

Trial name,status,loc,epochs,lr,n_layer,units,acc,iter,total time (s)
TrainH1stModel_5b283d1a,TERMINATED,,18,0.0320569,3,12,0.815556,1,1.75874
TrainH1stModel_5b286d44,TERMINATED,,42,0.0186211,5,4,0.813434,4,10.5708
TrainH1stModel_5b28988c,TERMINATED,,42,0.0490976,6,2,0.817172,1,2.74622
TrainH1stModel_5b28c29e,TERMINATED,,5,0.0469565,8,50,0.774747,1,2.28431
TrainH1stModel_5b28eb70,TERMINATED,,12,0.0251465,3,58,0.816667,1,1.71975
TrainH1stModel_5b2913b6,TERMINATED,,23,0.0178778,4,7,0.817172,1,1.99012
TrainH1stModel_5b293a76,TERMINATED,,4,0.0479921,2,7,0.811818,1,0.996375
TrainH1stModel_5b29628a,TERMINATED,,47,0.00769916,7,1,0.817374,5,15.6081
TrainH1stModel_5b298ada,TERMINATED,,32,0.0240371,4,7,0.816364,4,8.92108
TrainH1stModel_5b29bb04,TERMINATED,,45,0.0420289,2,39,0.812424,1,2.21712


In [11]:
analysis.dataframe()[['mean_accuracy', 'config/epochs','config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False)

,mean_accuracy,config/epochs,config/lr,config/n_layer,config/units
65,0.820707,22,0.042527,7,22
79,0.820404,20,0.029106,2,17
150,0.820404,6,0.007414,5,53
86,0.820303,15,0.023574,7,41
89,0.820303,41,0.006912,6,5
...,...,...,...,...,...
151,0.803333,9,0.010470,1,42
137,0.797374,11,0.005908,1,58
3,0.774747,5,0.046956,8,50
126,0.774747,49,0.038852,8,59


In [ ]:
analysis.dataframe()[['mean_accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False).head(20).mean_accuracy.mean()

In [12]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

 719.01 seconds,   11.98 minutes


## Bayesian Optimization (BO)

In [13]:
"""This test checks that BayesOpt is functional.

It also checks that it is usable with a separate scheduler.
"""
import time


import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.suggest.bayesopt import BayesOptSearch


# def evaluation_fn(step, width, height):
#     return (0.1 + width * step / 100)**(-1) + height * 0.1


# def easy_objective(config):
#     # Hyperparameters
#     width, height = config["width"], config["height"]

#     for step in range(config["steps"]):
#         # Iterative training function - can be any arbitrary training procedure
#         intermediate_score = evaluation_fn(step, width, height)
#         # Feed the score back back to Tune.
#         tune.report(iterations=step, mean_loss=intermediate_score)
#         time.sleep(0.1)

space = {
    "lr": tune.uniform(0.005, 0.05),
    "units": tune.uniform(1, 64),
    "epochs": tune.uniform(4, 50),    
#     "units": tune.randint(1, 64),
#     "n_layer": tune.randint(1, 8)
    "n_layer": tune.quniform(1, 8, 1.0),    
}

algo = BayesOptSearch(   
    utility_kwargs={
    "kind": "ucb",
    "kappa": 2.5,
    "xi": 0.0
}
)

algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

analysis= tune.run(
    TrainH1stModel,
    config=space,
    metric="mean_accuracy",
    mode="max",     
#     name="my_exp",
    search_alg=algo,
#     scheduler=scheduler,
    stop={"training_iteration": 5},
    verbose=1,
    num_samples=200
)


Trial name,status,loc,epochs,lr,n_layer,units,acc,iter,total time (s)
TrainH1stModel_08832bb8,TERMINATED,,21.2288,0.0477821,6.12396,38.7155,0.774747,5,12.2986
TrainH1stModel_088370dc,TERMINATED,,11.1769,0.0120198,1.40659,55.5691,0.805354,5,5.43821
TrainH1stModel_08839c24,TERMINATED,,31.6513,0.0368633,1.14409,62.1043,0.804343,5,7.44313
TrainH1stModel_0883cb4a,TERMINATED,,42.2924,0.0145553,2.27277,12.5545,0.81596,5,10.1319
TrainH1stModel_0883f566,TERMINATED,,17.9951,0.028614,4.02362,19.3474,0.815455,5,9.58377
TrainH1stModel_08841f14,TERMINATED,,32.1452,0.0112772,3.04501,24.0808,0.81404,5,10.4332
TrainH1stModel_08844822,TERMINATED,,24.9792,0.0403329,2.39772,33.3968,0.817879,5,6.77334
TrainH1stModel_088473a6,TERMINATED,,31.2511,0.00709027,5.25281,11.743,0.816263,5,9.13721
TrainH1stModel_08849d9a,TERMINATED,,6.99237,0.0476998,7.75942,51.929,0.774747,5,8.70692
TrainH1stModel_0884c842,TERMINATED,,18.0122,0.00939525,5.78963,28.7296,0.81404,5,8.58617


In [14]:
analysis.dataframe()[['mean_accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False).head(20)

,mean_accuracy,config/epochs,config/lr,config/n_layer,config/units
167,0.820808,48.471016,0.014146,2.354699,12.896976
150,0.820707,17.336949,0.038080,5.595966,22.170265
152,0.820707,25.253166,0.039597,2.267961,33.054504
180,0.820505,11.361999,0.027139,7.568089,15.114374
160,0.820505,5.150854,0.048557,7.067072,12.719363
70,0.820404,28.348225,0.042316,4.895986,10.608284
100,0.820303,30.565560,0.037953,6.987755,2.666633
164,0.820202,9.462350,0.012750,2.382344,7.208004
19,0.820202,38.159558,0.028629,3.102381,2.239624
198,0.820101,43.146149,0.049827,6.152356,11.950431


In [15]:
analysis.dataframe()[['mean_accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False).head(20).mean_accuracy.mean()

0.8201515151515151

In [16]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

 845.35 seconds,   14.09 minutes


### with AsyncHyperBandScheduler

In [ ]:
analysis.dataframe()[['mean_accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False).head(20)

In [ ]:
analysis.dataframe()[['mean_accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False).head(20).mean_accuracy.mean()

In [ ]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

## OptunaSearch

## HyperOptSearch